In [ ]:
import glob
import random
from functools import reduce
import json
from concurrent.futures import ThreadPoolExecutor
import pathlib as path

import pandas as pd
from scipy.spatial import KDTree
from collections import defaultdict
from argparse import Namespace
from scipy.spatial.qhull import QhullError
import tqdm
import numpy as np

import h5py as h5
from skimage.morphology import watershed, dilation, ball, closing, opening
from skimage.external.tifffile import imread
from skimage.measure import regionprops
from scipy import ndimage as ndi
from matplotlib import pyplot as plt

# Replication Data Analysis

In [ ]:
# get minimal size of all files
metadata_files = glob.glob('C:/Users/david/Desktop/replication-data/fixed_single_channel2/metadata*.json')

sizes = []
for file in metadata_files:
    with open(file, 'r') as fd:
        meta = json.load(fd)
    sizes.append([meta['voxel_size_z'], meta['voxel_size_y'], meta['voxel_size_x']])
    
min_size = np.min(np.array(sizes), axis=0)
min_size

In [ ]:
# segmentation

replication_overlap_thresh = 0.5 # how much of a replication site has to be inside the nuclear mask to keep it

def segment(marker_file, plot=False):

    # get segmentation/raw/metadata files corresponding to marker file
    prob_file = marker_file.replace('markers', 'classification_dapi')
    prob_file_edu = marker_file.replace('markers', 'classification_edu').replace('dapi', 'edu')
    prob_file_pcna = marker_file.replace('markers', 'classification_pcna').replace('dapi', 'pcna')
    
    raw_file = marker_file.replace('markers', '').replace('.h5', '.tif')
    raw_file_edu = marker_file.replace('markers', '').replace('.h5', '.tif').replace('dapi', 'edu')
    raw_file_pcna = marker_file.replace('markers', '').replace('.h5', '.tif').replace('dapi', 'pcna')
    
    metadata_file = marker_file.replace('markers', '').replace('.h5', '.json').replace('dapi', 'metadata')
    
    # get zoom factor necessary to have maximal isotropic resolution
    with open(metadata_file, 'r') as fd:
        meta = json.load(fd)
    size = np.array([meta['voxel_size_z'], meta['voxel_size_y'], meta['voxel_size_x']])
    zoom_factor = size/np.min(min_size)


    with h5.File(marker_file, 'r') as fd:
        markers = fd['markers'][...].squeeze()

    # read prob files
    with h5.File(prob_file, 'r') as fd:
        probs = fd['probs'][..., 1] # NB.: [...] converts to np-array
        probs_bg = fd['probs'][..., 0]
    with h5.File(prob_file_edu, 'r') as fd:
        probs_edu = fd['probs'][..., 1]        
    with h5.File(prob_file_pcna, 'r') as fd:
        probs_pcna = fd['probs'][..., 1]
        
    # read raw files
    raw = imread(raw_file)
    raw_edu = imread(raw_file_edu)
    raw_pcna = imread(raw_file_pcna)

    # introduce background marker 1: areas with high bg prob
    # all other labels += 1
    markers = ndi.label(markers)[0]
    markers[markers>0] += 1
    markers[probs_bg > 0.99] = 1 # TODO: make parameter?

    # watershed from markers -> dapi segmentation
    segmentation = watershed(probs, markers) #, compactness=0.0002) # TODO: compactness necessary?
    segmentation[segmentation==1] = 0
    
    # get the most central object
    center = np.array(segmentation.shape).astype(np.float)/2
    center_label = None
    center_dist = None
    for rprop in regionprops(segmentation):
        if center_label is None or np.linalg.norm(center-rprop.centroid) < center_dist:
            center_label = rprop.label
            center_dist = np.linalg.norm(center-rprop.centroid)

    seg_dapi = segmentation==center_label
                       
    # edu/pcna mask -> foreground prob higher than 0.5 
    seg_edu = (probs_edu > 0.5).astype(np.int)
    seg_pcna = (probs_pcna > 0.5).astype(np.int)
    
    # watershed on edt to separate replication sites
    #seg_edu *= watershed(-ndi.morphology.distance_transform_edt(seg_edu))
    seg_edu *= watershed(-ndi.gaussian_filter(ndi.morphology.distance_transform_edt(seg_edu), (0.5,1,1)))
    #seg_pcna *= watershed(-ndi.morphology.distance_transform_edt(seg_pcna))
    seg_pcna *= watershed(-ndi.gaussian_filter(ndi.morphology.distance_transform_edt(seg_pcna), (0.5,1,1)))
    
    # remove replication objects not within nuclear mask
    for rprop in regionprops(seg_edu):
        if np.sum(rprop.image * seg_dapi[rprop.slice]) < replication_overlap_thresh * np.sum(rprop.image):
            seg_edu[rprop.slice] = 0    
    for rprop in regionprops(seg_pcna):
        if np.sum(rprop.image * seg_dapi[rprop.slice]) < replication_overlap_thresh * np.sum(rprop.image):
            seg_pcna[rprop.slice] = 0
    
    if plot:
        fig, axs = plt.subplots(ncols=3, figsize=(15,5))
        axs[0].imshow(np.max(raw, axis=0), cmap='gray')
        axs[1].imshow(np.max(markers, axis=0), cmap='jet', interpolation='nearest')
        axs[2].imshow(np.max(seg_dapi, axis=0), cmap='gray', interpolation='nearest')
        [ax.axis('off') for ax in axs]
        [ax.set_title(t) for t,ax in zip(['DAPI', 'Markers', 'Central nucleus'], axs)]
        fig.tight_layout()
        
        fig, axs = plt.subplots(ncols=2, figsize=(10,5))
        axs[0].imshow(np.max(raw_edu, axis=0), cmap='gray')
        axs[1].imshow(np.max(seg_edu, axis=0), cmap='jet', interpolation='nearest')
        [ax.axis('off') for ax in axs]
        [ax.set_title(t) for t,ax in zip(['EdU', 'Replication sites'], axs)]
        fig.tight_layout()
        
        fig, axs = plt.subplots(ncols=2, figsize=(10,5))
        axs[0].imshow(np.max(raw_pcna, axis=0), cmap='gray')
        axs[1].imshow(np.max(seg_pcna, axis=0), cmap='jet', interpolation='nearest')
        [ax.axis('off') for ax in axs]
        [ax.set_title(t) for t,ax in zip(['PCNA', 'Replication sites'], axs)]
        fig.tight_layout()

    
    # upscale everything to maximum isotropic resolution
    raw = ndi.zoom(raw, zoom_factor)
    raw_edu = ndi.zoom(raw_edu, zoom_factor)
    raw_pcna = ndi.zoom(raw_pcna, zoom_factor)
    seg_dapi = ndi.zoom(seg_dapi, zoom_factor, order=0)
    seg_edu = ndi.zoom(seg_edu, zoom_factor, order=0)
    seg_pcna = ndi.zoom(seg_pcna, zoom_factor, order=0)
    
    return raw, raw_edu, raw_pcna, seg_dapi, seg_edu, seg_pcna

In [ ]:
# get files to process (marker files)
marker_dir = 'C:/Users/david/Desktop/replication-data/fixed_single_channel2/markers/'
marker_files = glob.glob(marker_dir + '*.h5')

In [ ]:
# test on one image with plot
#for marker_file in random.sample(marker_files, 1):

_ = segment(marker_files[1], True)

In [ ]:
# process all multithreaded    
with ThreadPoolExecutor() as tpe:
    futures = [tpe.submit(segment, marker_file) for marker_file in marker_files]
    results = [f.result() for f in tqdm.tqdm(futures)]

In [ ]:
def get_features(raw, raw_edu, raw_pcna, seg_dapi, seg_edu, seg_pcna, marker_file):
    
    # get stage and chase duration from filename
    stage = path.Path(marker_file).name.replace('.h5', '').split('_')[1]
    chase_dur = float(path.Path(marker_file).name.replace('.h5', '').split('_')[2].replace('h', ''))
    fname = path.Path(marker_file).name.replace('.h5', '')
    
    # we need edt on dapi for dsitance from border
    edt = ndi.morphology.distance_transform_edt(seg_dapi)
    
    # TODO: remove thin objects?
    #seg_edu = opening(seg_edu, ball(1))
    #seg_pcna = opening(seg_pcna, ball(1))

    # get 5% and 99% quantile for normalized intensities
    # TODO: make settable?
    q_min_dapi, q_max_dapi = np.quantile(raw[seg_dapi], [0.05, 0.99])
    q_min_edu, q_max_edu = np.quantile(raw_edu[seg_dapi], [0.05, 0.99])
    q_min_pcna, q_max_pcna = np.quantile(raw_pcna[seg_dapi], [0.05, 0.99])

    features_edu = defaultdict(list)
    centroids_edu = []    
    
    # go through all connected objects
    for rprop in regionprops(seg_edu):
        centroids_edu.append(rprop.centroid) # remember centroid -> we need it for nn analysis later

        # add all scalar regionprops
        for prop in rprop:
            try:
                if np.isscalar(rprop[prop]):
                    features_edu['edu_' + prop].append(rprop[prop])
            except NotImplementedError as e:
                pass
            except QhullError as e: # sometimes happens, add nan in that case, will be resolved later
                features_edu['edu_' + prop].append(np.nan)

        # add means in distance and intensity images
        means = Namespace()
        means.edt_mean = np.mean(edt[rprop.slice][rprop.image])    
        means.dapi_mean = np.mean(raw[rprop.slice][rprop.image])
        means.edu_mean = np.mean(raw_edu[rprop.slice][rprop.image])
        means.pcna_mean = np.mean(raw_pcna[rprop.slice][rprop.image])    
        means.dapi_mean_norm = (means.dapi_mean - q_min_dapi) / (q_max_dapi - (q_min_dapi))
        means.edu_mean_norm = (means.edu_mean - q_min_edu) / (q_max_edu - (q_min_edu))
        means.pcna_mean_norm = (means.pcna_mean - q_min_pcna) / (q_max_pcna - (q_min_pcna))
        for prop, m in means.__dict__.items():
            features_edu['edu_' + prop].append(m)

        features_edu['stage'].append(stage)
        features_edu['chase_dur'].append(chase_dur)
        features_edu['file'].append(fname)

    # same as above for pcna
    features_pcna = defaultdict(list)
    centroids_pcna = []
    for rprop in regionprops(seg_pcna):
        centroids_pcna.append(rprop.centroid)

        # add all scalar regionprops
        for prop in rprop:
            try:
                if np.isscalar(rprop[prop]):
                    features_pcna['pcna_' + prop].append(rprop[prop])
            except NotImplementedError as e:
                pass
            except QhullError as e:
                features_pcna['pcna_' + prop].append(np.nan)

        # add means
        means = Namespace()
        means.edt_mean = np.mean(edt[rprop.slice][rprop.image])    
        means.dapi_mean = np.mean(raw[rprop.slice][rprop.image])
        means.edu_mean = np.mean(raw_edu[rprop.slice][rprop.image])
        means.pcna_mean = np.mean(raw_pcna[rprop.slice][rprop.image])    
        means.dapi_mean_norm = (means.dapi_mean - q_min_dapi) / (q_max_dapi - (q_min_dapi))
        means.edu_mean_norm = (means.edu_mean - q_min_edu) / (q_max_edu - (q_min_edu))
        means.pcna_mean_norm = (means.pcna_mean - q_min_pcna) / (q_max_pcna - (q_min_pcna))
        for prop, m in means.__dict__.items():
            features_pcna['pcna_' + prop].append(m)

        features_pcna['stage'].append(stage)
        features_pcna['chase_dur'].append(chase_dur)
        features_pcna['file'].append(fname)


    n_edu = len(centroids_edu)
    n_pcna = len(centroids_pcna)

    # build kdtrees if we have detected objects
    if n_edu > 0:
        centroids_edu = np.array(centroids_edu)
        kd_edu = KDTree(centroids_edu)
    if n_pcna > 0:
        centroids_pcna = np.array(centroids_pcna)
        kd_pcna = KDTree(centroids_pcna)

    # which k-nn to use for distances
    # TODO: make settable
    ks = [1, 3, 5]

    # get mean nn-dsitances or nan if not enough neighbours
    for c_edu in centroids_edu:
        for k in ks:
            if n_pcna >= k:
                dst, _ = kd_pcna.query(c_edu, k)
                features_edu['edu_{}_nn_dist_pcna'.format(k)].append(np.mean(dst))
            else:
                features_edu['edu_{}_nn_dist_pcna'.format(k)].append(np.nan)
        for k in ks:
            if n_edu >= (k + 1):
                dst, _ = kd_edu.query(c_edu, k+1)
                features_edu['edu_{}_nn_dist_edu'.format(k)].append(np.mean(dst[1:]))
            else:
                features_edu['edu_{}_nn_dist_edu'.format(k)].append(np.nan)

    # same for pcna
    for c_pcna in centroids_pcna:
        for k in ks:
            if n_pcna >= (k + 1):
                dst, _ = kd_pcna.query(c_pcna, k+1)
                features_pcna['pcna_{}_nn_dist_pcna'.format(k)].append(np.mean(dst[1:]))
            else:
                features_pcna['pcna_{}_nn_dist_pcna'.format(k)].append(np.nan)
        for k in ks:
            if n_edu >= k:
                dst, _ = kd_edu.query(c_pcna, k)
                features_pcna['pcna_{}_nn_dist_edu'.format(k)].append(np.mean(dst))
            else:
                features_pcna['pcna_{}_nn_dist_edu'.format(k)].append(np.nan)

                
    # remove features for which we added nan in some cases (due to QHullError)
    if len(features_edu) > 0:
        max_len = np.max([len(v) for v in features_edu.values()])
        discard_keys = [k for k,v in features_edu.items() if len(v) != max_len]
        for k in discard_keys:
            del features_edu[k]

    if len(features_pcna) > 0:
        max_len = np.max([len(v) for v in features_pcna.values()])
        discard_keys = [k for k,v in features_pcna.items() if len(v) != max_len]
        for k in discard_keys:
            del features_pcna[k]
            
    # convex image might not be deleted
    # TODO: why?
    if 'edu_convex_image' in features_edu:
        del features_edu['edu_convex_image']
    if 'pcna_convex_image' in features_pcna:
        del features_pcna['pcna_convex_image']
    
    return features_edu, features_pcna

In [ ]:
# calculate features on single image

idx = 3
raw, raw_edu, raw_pcna, seg_dapi, seg_edu, seg_pcna = results[idx]
marker_file = marker_files[idx]
features_edu, features_pcna = get_features(raw, raw_edu, raw_pcna, seg_dapi, seg_edu, seg_pcna, marker_file)

In [ ]:
# compute features for all images
# TODO: multithreading?

features = []
for i, (result, marker_file) in enumerate(zip(results, marker_files)):
    features.append(get_features(*result, marker_file))
    print('processed {} of {}: {}'.format(i+1, len(marker_files), marker_file))

In [ ]:
# combine features for all images
def reduce_features(a, b):
    for k,v in b[0].items():
        a[0][k].extend(b[0][k])
    for k,v in b[1].items():
        a[1][k].extend(b[1][k])
    return a

pool_features_edu, pool_features_pcna = reduce(reduce_features, features, (defaultdict(list), defaultdict(list)))

In [ ]:
# make DataFrame

# calculate median, stddev, count for all features per cell
df_agg_edu = pd.DataFrame.from_dict(pool_features_edu).groupby(['file', 'stage', 'chase_dur']).agg(['median', 'count', 'std'])
df_agg_edu.columns = df_agg_edu.columns.to_flat_index() # collapse multi-index
df_agg_edu.columns = ['_'.join(c) for c in df_agg_edu.columns]
df_agg_edu['edu_num_sites'] = df_agg_edu['edu_area_count'] # copy one count column, call it num_sites
df_agg_edu = df_agg_edu.drop([c for c in df_agg_edu.columns if c.endswith('count')], 1) # remove all redundant count columns

# same for pcna
df_agg_pcna = pd.DataFrame.from_dict(pool_features_pcna).groupby(['file', 'stage', 'chase_dur']).agg(['median', 'count', 'std'])
df_agg_pcna.columns = df_agg_pcna.columns.to_flat_index()
df_agg_pcna.columns = ['_'.join(c) for c in df_agg_pcna.columns]
df_agg_pcna['pcna_num_sites'] = df_agg_pcna['pcna_area_count']
df_agg_pcna = df_agg_pcna.drop([c for c in df_agg_pcna.columns if c.endswith('count')], 1)

In [ ]:
# join edu & pcna features
df_agg = df_agg_edu.join(df_agg_pcna,  how='outer')

# remove columns with only one value, and explicitly euler_number columns (a few sites had euler!=1)
df_agg = df_agg.drop([c for c in df_agg.columns if len(df_agg[c].unique()) == 1 or ('euler_number' in c)], 1)
df_agg.head()

In [ ]:
# save
df_agg.to_csv('C:/Users/david/Desktop/replication-data/df_agg.csv')